<a id="top"></a>
<!-- # Pan-STARRS access using MAST's TAP service: How to chunk large / long queries -->
# Accessing MAST TAP services: How to chunk large / long queries

******

## Overview

This notebook demonstrates how to split up a large query (in terms of total number of rows returned or total execution time) for submission to Table Access Protocol (TAP) services at MAST. The examples presented here access the Pan-STARRS TAP service.

This includes how to scope out the query execution time and "row density" (number of rows for a given ID or spatial range), how to scale to appropriately-sized query chunks given the TAP service limitations, and how to structure the chunks with parallel and serial components (to avoid overusing database resources with too many queries in parallel).

## Learning Goals
By the end of this tutorial, you will:

- Understand how to **estimate the scaling of execution time and numbers of rows returned** with ID range or sky coverage
- Understand how to **use the scaling estimation to design full query chunks** that are compatible with TAP service limitations
- Understand how to submit the chunked queries in a manner that **does not overuse** database resources.

****
### Table of Contents

1. [Imports](#Imports)
1. [Connect to TAP service](#Connect-to-TAP-service)
1. [Too many rows](#Too-many-rows)
1. [Long query duration](#Long-query-duration)
1. [Additional Resources](#Additional-Resources)
1. [About This Notebook](#About-this-Notebook)

*******
## Introduction

Welcome! This notebook demonstrates how to perform large queries accessing the Pan-STARRS catalogs at MAST through a Virual Observatory Table Access Protocol (TAP) service, where **the full query has:**
1. **an execution time _longer_ than the time limit** for queries to the TAP service, and/or
2. **_more_ result rows than the maximum size of rows** returned by the TAP service.

In this tutorial, the example queries are drawn from the Pan-STARRS "20 queries" tutorials. Please see <b style="color:red">the 20 queries notebook [LINK HERE]</b> for full information about these queries.

The workflow for this notebook consists of:
* [Imports](#Imports)
* [Connect to TAP service](#Connect-to-TAP-service)
    * [TAP service limits](#TAP-service-limits)
* [Too many rows](#Too-many-rows)
    * [Determining the query row/time scaling](#Q2-query-scaling)
    * [Submitting the partitioned query](#Q2-submitting-query)
* [Long query duration](#Long-query-duration)
    * [Determining the query row/time scaling](#Q4-query-scaling)
    * [Submitting the partitioned query](#Q4-submitting-query)
* [Additional Resources](#Additional-Resources)
* [About This Notebook](#About-this-Notebook)

## Imports
This tutorial makes use of the following libraries: 
- [*numpy*](https://numpy.org/) for numerical calculations
- [*pyvo*](https://pyvo.readthedocs.io) for querying the MAST catalogs via TAP
- *time*, *datetime* to determine query duration
- [*astropy.table vstack*](https://docs.astropy.org/en/stable/table/index.html) to concatenate chunk result tables
- [*astropy.conf*](https://docs.astropy.org/en/stable/config/index.html) to modify number of lines displayed for tables

In [ ]:
import numpy as np
import pyvo as vo
import datetime
import time
from astropy.table import vstack

from astropy import conf
conf.max_lines = 10

--------
## Connect to TAP service

For all queries, we will be connecting to the Pan-STARRS (PS1) Data release 2 (DR2) catalog. Specifically, we will connect to the new [PS1 DR2 postgres-backed TAP service](https://mast.stsci.edu/vo-tap/api/v0.1/ps1_dr2/), which offers improved performance relative to the legacy database (by factors of 100 or greater, in many cases).

See the [PS1 documentation](link_to_migration_guide_here) for information about the tables available with this new TAP service.
    
<div class="alert alert-danger">
<b>FIX LINK TO MIGRATION GUIDE ABOVE</b>
</div>

In [ ]:
TAP_service = vo.dal.TAPService(
    "https://mast.stsci.edu/vo-tap/api/v0.1/ps1_dr2"
)

### TAP service limits

This service supports the following ADQL features, 
and will return up to **100,000 rows** --- which can be found in the TAP service description as seen below.

In [ ]:
TAP_service.describe()

The maximum query execution duration for this TAP service is **1200 seconds** (20 min).  The default job execution duration is 600 seconds.

*This can be found as follows:*
1. *Create a job with an empty query.*
2. *Set the execution duration to a very long time.*
3. *Check the job execution duration (set to the higher of the specified duration or the service maximum)*
4. *Delete the job*

In [ ]:
job = TAP_service.submit_job("")
job.execution_duration = int(1e7)   # seconds; Almost 1/3 of a year
print(job.execution_duration)
job.delete()

*********
*********

## Too many rows

First, we will examine Q2 from <b style="color:red">the Pan-STARRS "20 queries" [add link here]</b>, which looks for galaxies with a specified blue surface brightness within a fairly large swath of the sky (RA between 170 and 190 deg, and Decl. less than 0).

The full query is written as follows:

In [ ]:
full_adql_query = f"""
SELECT fmo.objID, fmo.gFKronMag,
   fmo.gFKronMag + 2.5*log10(PI()*POWER(so.gKronRad,2)) AS bsurfmag, 
   fmo.raMean, fmo.decMean
FROM forced_mean_object as fmo
JOIN stack_object AS so ON 
    fmo.objID=so.objID
WHERE ((fmo.gFKronMag > 0) AND
       (fmo.gFPSFMag - fmo.gFKronMag > 0.05)) -- galaxies
AND fmo.raMean BETWEEN 170 AND 190            -- ra substitute for super-gal coords
AND fmo.decMean < 0
AND fmo.gFKronMag + 2.5*log10(PI()*POWER(so.gKronRad,2))
       BETWEEN 23 AND 25                      -- mag per sq arcsec
AND so.primaryDetection = 1                   -- primary detection in stack_object"""
print(full_adql_query)

-----
<a id="Q2-query-scaling"></a>
### Determining the query row/time scaling

First, we need submit a test query with a restricted range in order to estimate how the query execution time and number of rows scales with ID range (or area).

To do this, we will place limits on the `objID`. 

We will leverage the fact that [Pan-STARRS object identifiers](https://outerspace.stsci.edu/spaces/Pan-STARRS/pages/298812384/PS1+Object+Identifiers) are determined from their position on the sky to determine an `objID` range. Since the first 5 digits of `objID` are determined from `floor((decl+90)/0.00833333)`, we can easily determine objID ranges from a declination range.

For this first test query, we will limit `objID` based on a declination range of -5$^{\circ}$$\leq$decMean$<$0$^{\circ}$.

In [ ]:
def get_objID_from_decl(decl):
    return np.int64(np.floor((decl+90)/0.00833333)*1e13)

In [ ]:
decl_range = [-5, 0]
objidrange = [get_objID_from_decl(decl) for decl in decl_range]

In [ ]:
test_adql_query = f"""{full_adql_query}
AND fmo.objID >= {objidrange[0]}           -- dec range subselection
AND fmo.objID < {objidrange[1]}"""
print(test_adql_query)

In [ ]:
start = time.time()
job = TAP_service.run_async(test_adql_query)
end = time.time()
print(f"Elapsed time: {str(datetime.timedelta(seconds=end-start))}")

In [ ]:
TAP_results = job.to_table()
TAP_results

**Note that the result table has *100,000 rows*, equal to the service maximum limit.**

It is almost certain that the results were truncated.

Thus, we will submit a second test query with a smaller declination limit range of -1$^{\circ}$$\leq$decMean$<$0$^{\circ}$.

*(The test query execution time is well under the service maximum, and thus not a concern for our partitioning).*

In [ ]:
decl_range = [-1, 0]
objidrange = [get_objID_from_decl(decl) for decl in decl_range]

In [ ]:
test_adql_query = f"""{full_adql_query}
AND fmo.objID >= {objidrange[0]}           -- dec range subselection
AND fmo.objID < {objidrange[1]}"""
print(test_adql_query)

In [ ]:
start = time.time()
job = TAP_service.run_async(test_adql_query)
end = time.time()
print(f"Elapsed time: {str(datetime.timedelta(seconds=end-start))}")

In [ ]:
TAP_results = job.to_table()
TAP_results

This second test query has fewer results than the maximum, indicating we have received all matching results.

Since the area of a 1 degree range of declination, for the same RA range, decreases away from the equator, all other 1 degree declination chunks from -30 deg to 0 deg likely have fewer objects. *(Though in general, the number of results will also scale with source density --- a concern for high density regions such as the galactic plane.)*

So in this case, we will proceed with **declination range chunks of [-30,-29), [-29,-28), ..., [-2, -1), [-1, 0.01).**  Note we slightly pad the upper limit (to account for the remaining digits in the objIDs), ensuring the objects very close to decl. ~ 0 deg are included.

----
<a id="Q2-submitting-query"></a>
### Submitting the partitioned query

With the query chunking scheme defined, we next proceed to submit the partitioned queries.

**Submitting all chunks in parallel (simultaneously) will overuse database resources.**  
<!-- Simultaneous queries may result in IP blocking, in order to ensure other users can access the database. -->

It is acceptable to submit a few (**$\boldsymbol{\sim}$5**) simultaneous queries to MAST's TAP services.

Thus, for our example query, we will structure our job submission as follows:
1. Generate all 1 deg. declination range query ADQL strings.
2. Loop over the full set of chunked queries: submit 5 queries simultaneously, wait for these to complete, and gather the results; the proceed to the next simultaneous set of queries.

As part of this process, we well also check that each query chunk returns FEWER than the maximum number of rows (to ensure no results are lost), and that each query chunk successfully executed.

A short helper function, defined below, will handle the process of submitting simultaneous queries and gathering the results, including checking for errors or potentially missing results:

In [ ]:
# Helper function to run simultaneous queries and gather the results:
def run_simultaneous_queries(
    queries, tap_service,
    sleep_interval_sec=10, maximum_rows=100000,
    execution_duration=None
):
    # Start simultaneous jobs:
    jobs = []
    for query in queries:
        # Initialize job
        job = tap_service.submit_job(query)

        # Set execution time, if specified:
        if execution_duration is not None:
            job.execution_duration = execution_duration

        # Run job and add to list:
        job.run()
        jobs.append(job)

    # Wait for all jobs to be finished. 
    # Based on pyvo AsyncTAPJob.wait()
    # https://pyvo.readthedocs.io/en/latest/_modules/pyvo/dal/tap.html#AsyncTAPJob.wait
    # Active phases:
    active_phases = {"QUEUED", "EXECUTING", "RUN", "COMPLETED", "ERROR", "UNKNOWN"}
    completed_phases = {"COMPLETED"}
    failed_phases = {"ABORTED", "ERROR"}

    while True:
        # Check all job phases:
        statuses = []
        for i, job in enumerate(jobs):
            job_phase = job.phase
            if job_phase not in active_phases:
                raise ValueError(
                    "Job no longer active!\n"
                    f"URL: {job.url}\n"
                    "Query:\n"
                    f"{job.query}"
                )

            if job_phase in failed_phases:
                raise ValueError(
                    f"Job status: {job_phase}\n"
                    f"URL: {job.url}\n"
                    "Query:\n"
                    f"{job.query}"
                )

            statuses.append(True if job_phase in completed_phases else False)

        # Break if all jobs are completed:
        if np.all(statuses):
            break

        # Otherwise, pause until next check
        time.sleep(sleep_interval_sec)

    # Collect results, and check that no query hit the maximum number of results
    results = []
    for job in jobs:
        result = job.fetch_result().to_table()
        if len(result) == maximum_rows:
            raise ValueError(
                "Query hit maximum number of results!\n"
                f"URL: {job.url}\n"
                "Query:\n"
                f"{job.query}"
            )
        results.append(result)

        # Delete the job:
        job.delete()

    return results

In [ ]:
# Define the full set of query chunks, with 1 deg. declination ranges:

full_decl_range = [-30, 0]
decl_step = 1

# Generate an array with declination values on the boundaries:
decl_bounds = np.arange(full_decl_range[0], full_decl_range[1]+decl_step, decl_step)
# List comprehension to make lists for each decl. chunk range:
decl_ranges = [[decl_bounds[i],decl_bounds[i+1]] for i in range(len(decl_bounds)-1)]

# Pad the upper limit on the last range, to account for other digits in the objIDs 
# for objects at decl = 0 deg.
decl_ranges[-1][1] = 0.01

# Generate chunk queries
queries = []
for decl_range in decl_ranges:
    objidrange = [get_objID_from_decl(decl) for decl in decl_range]
    chunk_adql_query = f"""{full_adql_query}
        AND fmo.objID >= {objidrange[0]}           -- dec range subselection
        AND fmo.objID < {objidrange[1]}"""
    queries.append(chunk_adql_query)

In [ ]:
# Submit all query chunks and gather results:

n_per_chunk = 5  # Number of queries run simultaneously
n_queries = len(queries)
results_all = []

# Get wall-time to run full set of query chunks
start = time.time()
for i in range(int(np.ceil(n_queries/n_per_chunk))):
    imin = i*n_per_chunk
    imax = min([n_queries, (i+1)*n_per_chunk])
    results = run_simultaneous_queries(
        queries[imin:imax], TAP_service,
        sleep_interval_sec=10, maximum_rows=100000
    )
    results_all.extend(results)
end = time.time()
print(f"Elapsed time for full query: {str(datetime.timedelta(seconds=end-start))}")

# Aggregate all results into one table:
# Concatenate individual tables:
TAP_results = vstack(results_all)

# Clear memory:
results_all = None
results = None

In [ ]:
TAP_results

By submitting 5 query chunks at a time, it takes about **8 minutes** to run all partitions of the full query, 
obtaining the full result set of about **1.07 million objects**.

<div class="alert alert-block alert-info">
<b><u>Tip:</u> <i>Ensuring robustness against individual query chunk failures</i></b>

Depending on the complexity and total time required to run a query, it would be beneficial to modify the above approach to 
ensure that results from successfully executed chunks are not lost in the event that another chunk query fails (due to, e.g., isolated timeout issues).

This could be accomplished by saving the individual chunk results to disk, and then only rerunning those chunks where the query was previously unsuccessful.
</div>

*********
*********

## Long query duration

Next, we will consider Q4 from <b style="color:red">the Pan-STARRS "20 queries" [add link here]</b>, which looks for galaxies with large galaxies with large red surface brightnes and high ellipticity.

The full query is written as follows:

In [ ]:
full_adql_query = """
SELECT so.objID, so.raMean, so.decMean, so.rKronRad, so.rKronMag,
   so.rKronMag + 2.5*log10(PI()*POWER(so.rKronRad,2)) as rsurfmag,
   smf.rSerAb, smf.rSerChisq,
   sqrt(1-power(smf.rSerAb,2)) as ellipticity
FROM stack_model_fit_ser AS smf
JOIN stack_object AS so ON 
    smf.objID=so.objID 
    AND smf.uniquePspsSTid=so.uniquePspsSTid
WHERE so.rKronRad BETWEEN 30 AND 60
  AND so.rKronMag > 0
  AND so.rKronMag + 2.5*log10(PI()*POWER(so.rKronRad,2)) < 24 -- mag per sq arcsec
  AND smf.rSerChisq > 0
  AND smf.rSerAb < sqrt(0.75)
  AND so.rpsfQfPerfect > 0.9
  AND so.nDetections > 3
  AND so.nr > 1"""
print(full_adql_query)

-----
<a id="Q4-query-scaling"></a>
### Determining the query row/time scaling

As before, we will submit a test query with a restricted range in order to estimate how the query execution time and number of rows scales.

Again, we will limit the `objID` range in this test query.

Since the first 5 digits of Pan-STARRS `objID` are determined by `floor((decl+90)/0.00833333)` (see the [Pan-STARRS object identifiers](https://outerspace.stsci.edu/spaces/Pan-STARRS/pages/298812384/PS1+Object+Identifiers) documentation), Pan-STARRS `objID`s fall within the estimated range 72000000000000000 to 216010000000000000 (using declination -30 to 90.01 deg, with padding to include the remaining objID digits --- thus ensuring all objects within -30 to 90 deg are included).

For this test query, we will restrict the `objID`s to between 129500000000000000 and 133500000000000000 (roughly 1/36th of the estimated full range).

In [ ]:
test_adql_query = f"""{full_adql_query}
  AND so.objID >= 129500000000000000     -- objID range subselection
  AND so.objID < 133500000000000000"""
print(test_adql_query)

In [ ]:
start = time.time()
job = TAP_service.run_async(test_adql_query)
end = time.time()
print(f"Elapsed time: {str(datetime.timedelta(seconds=end-start))}")

In [ ]:
TAP_results = job.to_table()
TAP_results

This test query takes about 40 seconds and returns 67 rows.

*(As this test query has fewer than the maximum number of rows, no results were omitted and we can thus use this to determine the query scaling.)*

Thus, executing the query over the full Pan-STARRS catalog (~36 times larger) would take about **24 minutes** and return about **2100 rows**.

While the estimated total is within the TAP service row limit, this full query **would take longer than the service time limit**.

Thus, we will split the full query in half, with **`objID` chunks of [72000000000000000, 144005000000000000), [144005000000000000, 216010000000000000)**

----
<a id="Q4-submitting-query"></a>
### Submitting the partitioned query

As before, we proceed to generate and submit the partitioned queries.

Since we will only use 2 chunks in this case, we will submit both simulaneously (because up to 5 parallel queries is reasonable given database resources).

However, we need to **increase the the execution duration to its maximum of 20 minutes**, to cover the estimated ~12 minute execution time per chunk.

We will use the previously-defined helper function to handle job submission, result gathering, and error checks.

In [ ]:
# Define the full set of query chunks, with the two objID ranges:
objID_ranges = [
    [72000000000000000, 144005000000000000],
    [144005000000000000, 216010000000000000]
    ]
# Generate chunk queries
queries = []
for objidrange in objID_ranges:
    chunk_adql_query = f"""{full_adql_query}
  AND so.objID >= {objidrange[0]}     -- objID range subselection
  AND so.objID < {objidrange[1]}"""
    queries.append(chunk_adql_query)

In [ ]:
# Submit all query chunks and gather results:

n_per_chunk = 2  # Number of queries run simultaneously
n_queries = len(queries)
results_all = []

# Get wall-time to run full set of query chunks
start = time.time()
for i in range(int(np.ceil(n_queries/n_per_chunk))):
    imin = i*n_per_chunk
    imax = min([n_queries, (i+1)*n_per_chunk])
    results = run_simultaneous_queries(
        queries[imin:imax], TAP_service,
        sleep_interval_sec=10, maximum_rows=100000,
        execution_duration=1200
    )
    results_all.extend(results)
end = time.time()
print(f"Elapsed time for full query: {str(datetime.timedelta(seconds=end-start))}")

# Aggregate all results into one table:
# Concatenate individual tables:
TAP_results = vstack(results_all)

# Clear memory:
results_all = None
results = None

In [ ]:
TAP_results

By submitting the 2 query chunks simultaneously, it takes about **6.5 minutes** to run both partitions of the full query, 
obtaining the full result set of **1745 objects**.

<div class="alert alert-block alert-info">
<b><u>Note:</u> <i>Intrinsic source density variations</i></b>

In this case, the full result set size is only about 80% of what we estimated, suggesting our test query selected objects in an ID range (equivalently, a portion of the sky) with a higher than average source density.

If the source density is known to be variable (with many selected objects in e.g., the galactic plane or nearby galaxies), submitting a test query covering a high density region/ID range will provide a conservative scaling estimate.  This will ensure none of the chunks go over the TAP service time/row limits.
</div>

----------

## Additional Resources

### Another chunking option for Pan-STARRS: `randomID`
Another option available for the Pan-STARRS catalogs (though not generally for other survey's catalogs) is to chunk 
queries by ranges of `randomID` from the ObjectThin table (and joins thereof). `randomID` is a random number between 0 and 1 assigned to each object, regardless of position on the sky.  

However, this tutorial presented approaches using object ID ranges (extensible to position ranges) to provide broader applicability to queries with other datasets.

### Table Access Protocol

- IVOA standard for RESTful web service access to tabular data
- http://www.ivoa.net/documents/TAP/

### Pan-STARRS 1 DR 2

- https://outerspace.stsci.edu/display/Pan-STARRS/

### Astronomical Query Data Language (2.0)

- IVOA standard for querying astronomical data in tabular format, with geometric search support
- http://www.ivoa.net/documents/latest/ADQL.html

### PyVO

- an affiliated package for [astropy](https://www.astropy.org/)
- find and retrieve astronomical data available from archives that support standard IVOA virtual observatory service protocols.
- https://pyvo.readthedocs.io/en/latest/index.html


### Full list of MAST/TAP services
- A full list of available MAST TAP services can be found at:
- https://mast.stsci.edu/vo-tap


## Citations
If you use `astropy` for published research, please cite the
authors. Follow these links for more information about citing `astropy`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)

If you use Pan-STARRS data accessed through MAST for published research, 
please include the following acknowledgements, found at the following links:

* [Acknowledging Pan-STARRS](https://archive.stsci.edu/publishing/mission-acknowledgements#section-895d38a0-86b3-4143-b521-6cc3312701f9)
* [Acknowledging MAST](https://archive.stsci.edu/gsc/mast_data_use.html)


## About this Notebook

**Author(s):**  Sedona Price<br>
**Keyword(s):** Tutorial, TAP, pyvo, ADQL, Pan-STARRS <br>
**First Published:** 2026-01-07 <br>
**Last Updated:** 2026-01-07
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/style-guides/master/guides/images/stsci-logo.png" alt="Space Telescope Logo" width="200px"/> 